In [1]:
# # XXX XXX WHAT THE FUCK IS GOING ON WHAT THE FUCK IS GOING ON WHAT THE FUCK IS GOING ON (all the embeddings are the same?!) XXX XXX
# # XXX turns out only some are fucked?
# import torch
# import safetensors
# import pydantic
# import safetensors.torch
# from typing import Literal, Optional, List, Dict, Tuple
# import json
# from pathlib import Path
# DATASETS_PATH = Path("/mnt/align3_drive/adrianoh/dl_final_project_embeddings")
# NFCORPUS_PATH = DATASETS_PATH / "nfcorpus"
# SCIDOCS_PATH = DATASETS_PATH / "scidocs"
# FIQA_PATH = DATASETS_PATH / "fiqa"
# ARGUANA_PATH = DATASETS_PATH / "arguana"

# class EmbeddingDataset(pydantic.BaseModel):
#     model_config = pydantic.ConfigDict(arbitrary_types_allowed=True)
#     embeddings: torch.Tensor
#     ids: List[str]
#     documents: List[str]

# [AFAIK DEPRECATED]
# # XXX shit might not match because different models use different tokenizers so the chunks are different... FUCK
# class EmbeddingLoader:
#     def __init__(self, root_path: Path, allow_embedding_equality: bool = False):
#         self.root_path = root_path
#         self.allow_embedding_equality = allow_embedding_equality

#     def fetch_embeddings(
#         self,
#         corpus_folder: Literal["corpus", "query"] = "corpus",
#         model_folders: Optional[List[str]] = None
#     ) -> List[EmbeddingDataset]:
#         """
#         Fetch the set of embeddings for the given corpus and subfolders.
#         If it's `None` for `model_folders`, it will fetch all the models' embeddings.
#         """
#         if model_folders is None:
#             model_folders = [model_folder.name for model_folder in (self.root_path / corpus_folder).iterdir() if model_folder.is_dir()] # fmt: skip
#         subpaths = [(self.root_path / corpus_folder / model_folder) for model_folder in model_folders]
#         assert len(set(subpath.name for subpath in subpaths)) == len(subpaths)
#         # The files we want
#         embeddings_paths = [subpath / "embeddings.safetensors" for subpath in subpaths]
#         ids_paths = [subpath / "ids.json" for subpath in subpaths]
#         documents_paths = [subpath / "documents.json" for subpath in subpaths]
#         chunks_paths = [subpath / "chunks.json" for subpath in subpaths]

#         # They should all have the correct files present
#         assert all(embeddings_path.exists() for embeddings_path in embeddings_paths), ("Not all embeddings paths exist:\n  " + '\n  '.join(f'{p.as_posix()}: {p.exists()}' for p in embeddings_paths)) # fmt: skip
#         assert all(ids_path.exists() for ids_path in ids_paths), f"Not all ids paths exist: {ids_paths}"
#         assert all(documents_path.exists() for documents_path in documents_paths), f"Not all documents paths exist: {documents_paths}" # fmt: skip
#         assert all(chunks_path.exists() for chunks_path in chunks_paths), f"Not all chunks paths exist: {chunks_paths}" # fmt: skip

#         _embeddings_sets: List[Dict[str, torch.Tensor]] = [safetensors.torch.load_file(embeddings_path, device="cpu") for embeddings_path in embeddings_paths] # fmt: skip
#         _ids_sets: List[Dict[str, List[str]]] = [json.load(open(ids_path, "r")) for ids_path in ids_paths]
#         _documents_sets: List[Dict[str, List[str]]] = [json.load(open(documents_path, "r")) for documents_path in documents_paths] # fmt: skip
#         _chunks_sets: List[Dict[str, List[str]]] = [json.load(open(chunks_path, "r")) for chunks_path in chunks_paths] # fmt: skip

#         # Extrat the jsons
#         assert all(isinstance(ids_set, dict) and "ids" in ids_set and len(ids_set) == 1 for ids_set in _ids_sets)
#         assert all(isinstance(documents_set, dict) and "documents" in documents_set and len(documents_set) == 1 for documents_set in _documents_sets) # fmt: skip
#         assert all(isinstance(embeddings_set, dict) and "embeddings" in embeddings_set and len(embeddings_set) == 1 for embeddings_set in _embeddings_sets) # fmt: skip
#         assert all(isinstance(chunks_set, dict) and "chunks" in chunks_set and len(chunks_set) == 1 for chunks_set in _chunks_sets) # fmt: skip
#         ids_sets: List[List[str]] = [ids_set["ids"] for ids_set in _ids_sets]
#         documents_sets: List[List[str]] = [documents_set["documents"] for documents_set in _documents_sets] # fmt: skip
#         embeddings_sets: List[torch.Tensor] = [embeddings_set["embeddings"] for embeddings_set in _embeddings_sets] # fmt: skip
#         chunks_sets: List[List[str]] = [chunks_set["chunks"] for chunks_set in _chunks_sets] # fmt: skip

#         # Make sure that all these have the same lengths
#         _ids_lengths = [len(ids_set) for ids_set in ids_sets]
#         _documents_lengths = [len(documents_set) for documents_set in documents_sets] # fmt: skip
#         _embeddings_lengths = [len(embeddings_set) for embeddings_set in embeddings_sets] # fmt: skip
#         _chunks_lengths = [len(chunks_set) for chunks_set in chunks_sets] # fmt: skip
#         _nl = "\n"
#         assert len(subpaths) == len(_ids_lengths) == len(_documents_lengths) == len(_embeddings_lengths) == len(_chunks_lengths), f"Got different lengths: {_nl.join(f'{i.as_posix()}: {l}' for i, l in zip(subpaths, _ids_lengths, _documents_lengths, _embeddings_lengths, _chunks_lengths))}"
#         assert len(set(_ids_lengths)) == 1, f"Got different ids lengths: {_nl.join(f'{i.as_posix()}: {l}' for i, l in zip(subpaths, _ids_lengths))}"
#         assert len(set(_documents_lengths)) == 1, f"Got different documents lengths: {_nl.join(f'{i.as_posix()}: {l}' for i, l in zip(subpaths, _documents_lengths))}"
#         assert len(set(_embeddings_lengths)) == 1, f"Got different embeddings lengths: {_nl.join(f'{i.as_posix()}: {l}' for i, l in zip(subpaths, _embeddings_lengths))}"
#         assert len(set(_chunks_lengths)) == 1, f"Got different chunks lengths: {_nl.join(f'{i.as_posix()}: {l}' for i, l in zip(subpaths, _chunks_lengths))}"

#         # Make sure the types are OK => they come from the same dataset but different models embedded so they should ONLY differ in that
#         assert all(isinstance(ids_set, list) and all(isinstance(id, str) for id in ids_set) for ids_set in ids_sets) # fmt: skip
#         assert all(ids_set1 == ids_sets[0] for ids_set1 in ids_sets) # star pattern for equality
#         # ...
#         assert all(isinstance(documents_set, list) and all(isinstance(doc, str) for doc in documents_set) for documents_set in documents_sets) # fmt: skip
#         assert all(documents_set1 == documents_sets[0] for documents_set1 in documents_sets) # star pattern for equality
#         #...
#         assert all(isinstance(chunks_set, list) and all(isinstance(chunk, str) for chunk in chunks_set) for chunks_set in chunks_sets) # fmt: skip
#         assert all(chunks_set1 == chunks_sets[0] for chunks_set1 in chunks_sets) # star pattern for equality
#         # ...
#         # (only the first dimension should match)
#         assert all(isinstance(embeddings_set, torch.Tensor) and embeddings_set.shape[0] == embeddings_sets[0].shape[0] for embeddings_set in embeddings_sets) # fmt: skip
#         # Cartesian product: all most be unique
#         # avoid double-counting and self-comparison
#         if not self.allow_embedding_equality:
#             is_close: Dict[str, bool] = {}
#             for i in range(len(embeddings_sets)):
#                 for j in range(i + 1, len(embeddings_sets)):
#                     path1, path2 = embeddings_paths[i].parent.name, embeddings_paths[j].parent.name
#                     path_key = f"{path1} <|> {path2}"
#                     is_close[path_key] = torch.allclose(embeddings_sets[i], embeddings_sets[j])
#             is_close = {x : y for x, y in is_close.items() if y}
#             assert len(is_close) == 0, f"Got {len(is_close)} embeddings (total n_embeddings={len(embeddings_sets)}) that are the same:\n\n{json.dumps(is_close, indent=2)}"
#         return [
#             EmbeddingDataset(
#                 embeddings=embeddings_set,
#                 ids=ids_set,
#                 documents=documents_set
#             )
#             for embeddings_set, ids_set, documents_set in zip(embeddings_sets, ids_sets, documents_sets)
#         ]

# loader = EmbeddingLoader(root_path=NFCORPUS_PATH, allow_embedding_equality=False) # XXX this (the fact we allow True) is really unacceptable - something is WRONG and VERY WRONG ngl
# embeddings_list: List[EmbeddingDataset] = loader.fetch_embeddings(corpus_folder="corpus", model_folders=["intfloat_e5-large-v2", "thenlper_gte-small"]) # get all the document/corpus embeddings
# print("Embeddings shape is", embeddings_list[0].embeddings.shape, "=", embeddings_list[1].embeddings.shape, "= ...") # axis 0 is dataset, axis 1 is embedding
# print("Embeddings devices:", "embeddings1.device", embeddings_list[0].embeddings.device, "embeddings2.device", embeddings_list[1].embeddings.device, "etc...") # shoulds be cpu cpu



In [19]:
"""
Define the necessary pydantic structures where we will store everything. Everything will be in:
    <model1_folderable_name>_<model2_folderable_name>/
        <dataset_name>/
            # Linear transform
            linear_transform.safetensors
            stitch_info.json
            # Embeddings (generated via linear(model1(texts))) where linear maps from model1 space to model2 space
            embeddings_corpus_train.safetensors
            embeddings_corpus_validation.safetensors
            embeddings_query_train.safetensors
            embeddings_query_validation.safetensors
            # Metadatas
            metadatas_corpus_train.json
            metadatas_corpus_validation.json
            metadatas_query_train.json
            metadatas_query_validation.json
"""
import torch.nn as nn
from pydantic import BaseModel
import torch
import safetensors
import pydantic
import safetensors.torch
from typing import Literal, Optional, List, Dict, Tuple
import json
from pathlib import Path

class StitchPair(BaseModel):
    source: str
    target: str
    dataset: str
    mode: Literal["affine"] = "affine"  # TODO(Adriano) later we will support more shit here

    def save_linear_transform(self, linear: nn.Linear, save_path: Path) -> None:
        linear_path = save_path / "linear_transform.safetensors"
        stitch_info_path = save_path / "stitch_info.json"
        assert not linear_path.exists(), f"Linear transform already exists at {linear_path}"
        assert not stitch_info_path.exists(), f"Stitch info already exists at {stitch_info_path}"
        safetensors.torch.save_file(linear.state_dict(), linear_path)
        stitch_info_path.write_text(self.model_dump_json())

In [26]:
# Now, we can try to train linear transforms between embeddings...
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import tqdm
import torch.optim as optim
from pydantic import BaseModel
import wandb
from typing import Optional

class EmbeddingDataset(Dataset):
    def __init__(self, source_embeddings: torch.Tensor, target_embeddings: torch.Tensor):
        assert source_embeddings.shape[0] == target_embeddings.shape[0]
        self.source_embeddings = source_embeddings
        self.target_embeddings = target_embeddings
        
    def __len__(self):
        return len(self.source_embeddings)
        
    def __getitem__(self, idx):
        return self.source_embeddings[idx], self.target_embeddings[idx]

class LinearTransformTrainerArgs(BaseModel):
    # We use default for now :)
    test_split: float = 0.2
    num_epochs: int = 50
    batch_size: int = 32
    learning_rate: float = 0.001
    save_every_n_epochs: int = 10
    use_tqdm: bool = True

class LinearTransformTrainer:
    def __init__(
        self,
        save_path: Path,
        linear: Optional[nn.Linear],
        source_embeddings_train: torch.Tensor,
        target_embeddings_train: torch.Tensor,
        source_embeddings_validation: torch.Tensor,
        target_embeddings_validation: torch.Tensor,
        device: torch.device | str,
        args: LinearTransformTrainerArgs = LinearTransformTrainerArgs()
    ):
        self.linear = linear
        self.source_embeddings_train = source_embeddings_train
        self.target_embeddings_train = target_embeddings_train
        self.source_embeddings_validation = source_embeddings_validation
        self.target_embeddings_validation = target_embeddings_validation
        self.num_epochs = args.num_epochs
        self.batch_size = args.batch_size
        self.learning_rate = args.learning_rate
        self.device = device
        self.test_split = args.test_split
        self.save_every_n_epochs = args.save_every_n_epochs
        self.save_path = save_path
        self.checkpoint_path = save_path / "checkpoints"
        self.checkpoint_path.mkdir(parents=True, exist_ok=True)
        self.logfile = save_path / "log.jsonl"
        self.use_tqdm = args.use_tqdm
        if self.linear is None:
            self.linear = self.create_linear_transform()
        self.optimizer = torch.optim.Adam(self.linear.parameters(), lr=self.learning_rate)

    #### PRE-TRAINING HELPERS ####
    def create_datasets(self):
        # Create indices and shuffle
        # num_samples = len(self.source_embeddings_train)
        # indices = torch.randperm(num_samples)
        
        # Split indices -> NOTE we already created this shit
        # split_idx = int(num_samples * (1 - self.test_split))
        # train_indices = indices[:split_idx]
        # test_indices = indices[split_idx:]
        
        # Create datasets
        train_dataset = EmbeddingDataset(
            self.source_embeddings_train,
            self.target_embeddings_train
        )
        test_dataset = EmbeddingDataset(
            self.source_embeddings_validation,
            self.target_embeddings_validation
        )
        
        return train_dataset, test_dataset

    def create_linear_transform(self):
        return nn.Linear(self.source_embeddings_train.shape[1], self.target_embeddings_train.shape[1]).to(self.device)

    #### TRAINING HELPERS ####
    def train(self):
        train_dataset, test_dataset = self.create_datasets()
        train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=self.batch_size)

        mse_loss = nn.MSELoss()
        trange = tqdm.trange if self.use_tqdm else range
        tqdm_fn = tqdm.tqdm if self.use_tqdm else lambda *args, **kwargs: args[0]
        for epoch in trange(self.num_epochs):
            # Training
            self.linear.train()
            train_mse = 0.0
            train_mae = 0.0
            num_train_batches = 0

            for source_emb, target_emb in train_loader:
                source_emb = source_emb.to(self.device)
                target_emb = target_emb.to(self.device)

                self.optimizer.zero_grad()
                output = self.linear(source_emb)

                loss = mse_loss(output, target_emb)
                loss.backward()
                self.optimizer.step()

                train_mse += loss.detach().item()
                train_mae += (output.detach() - target_emb.detach()).abs().mean().item()
                num_train_batches += 1

            avg_train_mse = train_mse / num_train_batches
            avg_train_mae = train_mae / num_train_batches

            # Evaluation
            self.linear.eval()
            test_mse = 0.0
            test_mae = 0.0
            num_test_batches = 0

            with torch.no_grad():
                for source_emb, target_emb in tqdm_fn(test_loader):
                    source_emb = source_emb.to(self.device)
                    target_emb = target_emb.to(self.device)

                    output = self.linear(source_emb)

                    test_mse += mse_loss(output, target_emb).item()
                    test_mae += (output.detach() - target_emb.detach()).abs().mean().item()
                    num_test_batches += 1

            avg_test_mse = test_mse / num_test_batches
            avg_test_mae = test_mae / num_test_batches

            # Log metrics
            log_entry = {
                "epoch": epoch,
                "train_mse": avg_train_mse,
                "train_mae": avg_train_mae,
                "test_mse": avg_test_mse,
                "test_mae": avg_test_mae,
            }
            wandb.log(log_entry)
            with open(self.logfile, "a") as f:
                f.write(json.dumps(log_entry) + "\n")

            if epoch % self.save_every_n_epochs == 0:
                self.save_checkpoint(epoch)

    def save_checkpoint(self, epoch: int):
        checkpoint_path = self.checkpoint_path / f"checkpoint_{epoch}.safetensors"
        safetensors.torch.save_file(self.linear.state_dict(), checkpoint_path)

In [27]:
DEFAULT_ARGS = LinearTransformTrainerArgs(
    test_split=0.2,
    num_epochs=50,
    batch_size=32,
    learning_rate=0.001,
    save_every_n_epochs=10,
    use_tqdm=True
)

In [7]:
"""
Single layer train attempt. This should help us get some good data.
"""
import click
import os
import shutil
from safetensors.torch import load_file
save_path_parent = Path("/mnt/align3_drive/adrianoh/dl_final_project_layers")
embeddings1_path = Path("/mnt/align3_drive/adrianoh/dl_final_project_embeddings_huggingface/intfloat_e5-base-v2/arguana")
embeddings2_path = Path("/mnt/align3_drive/adrianoh/dl_final_project_embeddings_huggingface/thenlper_gte-base/arguana")
# work only on corpus right now
embeddings1_train = load_file(embeddings1_path / "embeddings_corpus_train.safetensors")["embeddings"] # X
embeddings1_validation = load_file(embeddings1_path / "embeddings_corpus_validation.safetensors")["embeddings"] # Y
embeddings2_train = load_file(embeddings2_path / "embeddings_corpus_train.safetensors")["embeddings"] # x
embeddings2_validation = load_file(embeddings2_path / "embeddings_corpus_validation.safetensors")["embeddings"] # y
assert embeddings1_train.shape[0] == embeddings2_train.shape[0]
assert embeddings1_validation.shape[0] == embeddings2_validation.shape[0]

save_path = save_path_parent / "ipynb_test"
assert os.environ.get("CUDA_VISIBLE_DEVICES") is None
if save_path.exists():
    click.echo(f"Save path {save_path} already exists. Deleting it...")
    # click.confirm(f"Save path {save_path} already exists. Do you want to delete it?", abort=True)
    shutil.rmtree(save_path)
device = "cuda:0" # change this based on availability
wandb.init(project="2024_12_09_dl_project_testing_layer_train", name="ipynb_test_dec_10")
trainer = LinearTransformTrainer(
    save_path=save_path, # made by the trainer
    linear=None, # trainer makes it
    source_embeddings_train=embeddings1_train.to(device),
    target_embeddings_train=embeddings2_train.to(device),
    source_embeddings_validation=embeddings1_validation.to(device),
    target_embeddings_validation=embeddings2_validation.to(device),
    device=device,
    args=DEFAULT_ARGS
)
trainer.train()
wandb.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: 4gate. Use `wandb login --relogin` to force relogin


100%|██████████| 50/50 [00:12<00:00,  3.87it/s]


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
test_mae,█▄▃▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
test_mse,█▄▂▂▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mae,█▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mse,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
test_mae,0.00841
test_mse,0.00011
train_mae,0.00799
train_mse,0.0001


In [58]:
# [AFAIK DEPRECATED] => all ingestion settings etc... are default
# class EmbeddingMetadata(pydantic.BaseModel):
#     """ChromaDB embedding metadata.

#     Every embedding in a ChromaDB collection has a metadata object that explains
#     basically whether its a query or a document.
#     """

#     record_id: str
#     chunk_id: str
#     chunk_text: str
#     record_text: str | None = None  # <---- like never used tbh
#     record_type: Literal["query", "document"]
#     # TODO(Adriano) not everything should be train by default
#     record_split: Literal["train", "test"] = "train"
#     tags: dict[str, str] | None = {}  # <---- should insert some meaning tags for umap cluster

# class IngestionSettings(BaseModel):
#     """Equivalent to owler's .env file.

#     Used to define how to process the ingestion of the textual data into chromaDB
#     """

#     chunk_size: int = 256
#     device: str | None = None  # does not matter
#     distance_function: str | None = None  # does not matter
#     normalize_embeddings: bool | None = None  # does not matter
#     # TODO(Adriano) in the future we will want to try passing this through a model before
#     chunk_preprocessing_mode: Literal["add_prefix"] = "add_prefix"
#     query_preprocessing_mode: Literal["add_prefix"] = "add_prefix"
#     chunk_prefix: str = (
#         "passage: "  # Can be used to add prefix to text embeddings stored in vector store
#     )
#     query_prefix: str = (
#         "query: "  # Can be used to add prefix to text embeddings used for semantic search
#     )
#     chunk_overlap: int = 25  # Determines, for a given chunk of text, how many tokens must overlap with adjacent chunks.
#     dataloader_batch_size: int = 32
#     dataloader_num_workers: int = 4

In [34]:
# class ModelMetadata(BaseModel):
#     """Metadata for a single model's embeddings."""
#     model: str
#     dataset: str = "default"
#     chunk_size: int = 256
#     query_prefix: str | None = None
#     document_prefix: str | None = None

# class TransformMetadata(BaseModel):
#     """Metadata for a transform between two models."""
#     source_model: ModelMetadata
#     target_model: ModelMetadata
import itertools
DATASETS = [
    # (numbers are counts for documents, there may be some longer documents -> slightly more chunks)
    "arguana", # 10K
    "fiqa", # 50K -> 20K
    "scidocs", # 25K -> 20K
    "nfcorpus", # 5K
    "hotpotqa", # 100K -> 20K
    "trec-covid", # too much -> 20K
]

MODEL_NAMES = [
    "Salesforce/SFR-Embedding-Mistral",
    "WhereIsAI/UAE-Large-V1",
    "BAAI/bge-base-en-v1.5",
    "BAAI/bge-large-en-v1.5",
    "BAAI/bge-small-en-v1.5",
    "intfloat/e5-base-v2",
    "intfloat/e5-large-v2",
    "intfloat/e5-small-v2",
    "thenlper/gte-base",
    "thenlper/gte-large",
    "thenlper/gte-small",
    "sentence-transformers/gtr-t5-base",
    "sentence-transformers/gtr-t5-large",
    "mixedbread-ai/mxbai-embed-large-v1",
    "sentence-transformers/sentence-t5-base",
    "sentence-transformers/sentence-t5-large",
    "text-embedding-3-large", # openai
    "text-embedding-3-small", # openai
]

class EmbeddingTransformTrainer:
    """Handles training transforms between embedding models."""
    
    def __init__(self, save_path_parent: Path, load_path_parent: Path, device: str, wandb_project: str):
        self.save_path_parent = save_path_parent
        self.load_path_parent = load_path_parent
        self.device = device
        self.wandb_project = wandb_project
        
    def train_all_pairs(
        self,
        filter_against_model: List[str] = [],
        filter_for_model: List[str] = [],
        filter_for_dataset: List[str] = [],
        filter_against_dataset: List[str] = []
    ):
        """Train transforms between all pairs of embeddings."""
        # filtering...
        combos = list(itertools.product(DATASETS, MODEL_NAMES, MODEL_NAMES))
        combos = [x for x in combos if x[1] != x[2]]
        # 1. filter for models
        for filter_against_model in filter_against_model:
            combos = [x for x in combos if filter_against_model not in x[1] and filter_against_model not in x[2]]
        for filter_for_model in filter_for_model:
            combos = [x for x in combos if filter_for_model in x[1] and filter_for_model in x[2]]
        # 2. filter for datasets
        for filter_for_dataset in filter_for_dataset:
            combos = [x for x in combos if filter_for_dataset in x[0]]
        for filter_against_dataset in filter_against_dataset:
            combos = [x for x in combos if filter_against_dataset not in x[0]]
        # End filtering...
        print(f"Training {len(combos)} transforms")
        for dataset, src, dest in tqdm.tqdm(combos, desc="Training transforms (all default settings)"):
            assert src != dest
            # plz plz
            src_name_ok = src.replace("/", "_")
            dest_name_ok = dest.replace("/", "_")
            dataset_ok = dataset.replace("/", "_")

            # 1. make sure shit is present in load
            embeddings1_path = self.load_path_parent / src_name_ok / dataset_ok
            embeddings2_path = self.load_path_parent / dest_name_ok / dataset_ok
            # NOTE ====> only corpus right now
            embeddings1_train_path = embeddings1_path / "embeddings_corpus_train.safetensors"
            embeddings1_validation_path = embeddings1_path / "embeddings_corpus_validation.safetensors"
            embeddings2_train_path = embeddings2_path / "embeddings_corpus_train.safetensors"
            embeddings2_validation_path = embeddings2_path / "embeddings_corpus_validation.safetensors"
            if any(not x.exists() for x in [embeddings1_train_path, embeddings1_validation_path, embeddings2_train_path, embeddings2_validation_path]):
                print(f"Skipping {src} to {dest} because some files do not exist in {embeddings1_path.name} or {embeddings2_path.name}")
                for file in [embeddings1_train_path, embeddings1_validation_path, embeddings2_train_path, embeddings2_validation_path]:
                    if file.exists():
                        print(f"File {file} exists")
                    else:
                        print(f"File {file} does not exist")
                continue

            # 2. make sure shit is NOT present in save
            save_parent = self.save_path_parent / f"{src_name_ok}_{dest_name_ok}" / dataset_ok
            stitch_info_pair_file = save_parent / "stitch_info_pairs.jsonl"
            linear_transform_file = save_parent / "linear_transform.safetensors"
            assert not linear_transform_file.exists()
            assert not stitch_info_pair_file.exists()
            save_parent.mkdir(parents=True, exist_ok=True)
            stitch_info_pair = StitchPair(
                source=src,
                target=dest,
                dataset=dataset
            )
            stitch_info_pair_file.write_text(stitch_info_pair.model_dump_json())

            # 3. load
            embeddings1_train = load_file(embeddings1_train_path)["embeddings"] # X
            embeddings1_validation = load_file(embeddings1_validation_path)["embeddings"] # Y
            embeddings2_train = load_file(embeddings2_train_path)["embeddings"] # x
            embeddings2_validation = load_file(embeddings2_validation_path)["embeddings"] # y
            # Initialize wandb run
            wandb.init(
                project=self.wandb_project,
                name=f"{src_name_ok}_{dest_name_ok}",
                reinit=True
            )
            # Train the model
            trainer = LinearTransformTrainer(
                save_path=save_path,
                linear=None,
                source_embeddings_train=embeddings1_train.to(self.device),
                target_embeddings_train=embeddings2_train.to(self.device),
                source_embeddings_validation=embeddings1_validation.to(self.device),
                target_embeddings_validation=embeddings2_validation.to(self.device),
                device=self.device,
                args=DEFAULT_ARGS
            )
            trainer.train()
            wandb.finish()

In [35]:
trainer = EmbeddingTransformTrainer(
    save_path_parent=Path("/mnt/align3_drive/adrianoh/dl_final_project_layers/arguana_hf_cartesian_product"),
    load_path_parent=Path("/mnt/align3_drive/adrianoh/dl_final_project_embeddings_huggingface"),
    device="cuda:0",
    wandb_project="2024_12_11_dl_project_layer_arguana_hf_only_train"
)
trainer.train_all_pairs(
    filter_against_model=["Salesforce/SFR-Embedding-Mistral","text-embedding-3-large","text-embedding-3-small"],
    filter_for_dataset=["arguana"],
)


Training 210 transforms


Training transforms (all default settings):   0%|          | 0/210 [00:00<?, ?it/s]


100%|██████████| 70/70 [00:00<00:00, 3146.55it/s]

100%|██████████| 70/70 [00:00<00:00, 3099.54it/s]

100%|██████████| 70/70 [00:00<00:00, 3169.41it/s]

100%|██████████| 70/70 [00:00<00:00, 3107.81it/s]

100%|██████████| 70/70 [00:00<00:00, 3096.57it/s]

100%|██████████| 70/70 [00:00<00:00, 3130.04it/s]

100%|██████████| 70/70 [00:00<00:00, 3147.70it/s]

100%|██████████| 70/70 [00:00<00:00, 3097.48it/s]

100%|██████████| 70/70 [00:00<00:00, 3099.87it/s]

100%|██████████| 70/70 [00:00<00:00, 3083.27it/s]

100%|██████████| 70/70 [00:00<00:00, 3168.48it/s]

100%|██████████| 70/70 [00:00<00:00, 3122.69it/s]

100%|██████████| 70/70 [00:00<00:00, 3157.44it/s]

100%|██████████| 70/70 [00:00<00:00, 3116.69it/s]

100%|██████████| 70/70 [00:00<00:00, 3116.22it/s]

100%|██████████| 70/70 [00:00<00:00, 3179.46it/s]

100%|██████████| 70/70 [00:00<00:00, 3093.70it/s]

100%|██████████| 70/70 [00:00<00:00, 3122.82it/s]

100%|██████████| 70/70 [00:00<00:00, 3118.04it/s]

100%|██████████| 70/70 [00:00<

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
test_mae,█▄▃▃▂▃▂▂▂▁▂▁▁▂▁▁▁▁▁▁▂▁▁▂▁▁▁▂▂▂▂▂▂▂▂▂▂▂▁▂
test_mse,█▄▃▃▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▁▂▁▁▂▂▁▁▂▂▂▂▂▂▂▁▂
train_mae,█▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mse,█▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
test_mae,0.01251
test_mse,0.00025
train_mae,0.01162
train_mse,0.00021


Training transforms (all default settings):   0%|          | 1/210 [00:15<54:29, 15.64s/it]/mnt/align3_drive/adrianoh/miniconda3/envs/python311/lib/python3.11/site-packages/wandb/sdk/wandb_run.py:2368: UserWarning: Run (4zvzgqqg) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),


...


100%|██████████| 70/70 [00:00<00:00, 3028.03it/s]

100%|██████████| 70/70 [00:00<00:00, 3016.18it/s]

100%|██████████| 70/70 [00:00<00:00, 2973.72it/s]

100%|██████████| 70/70 [00:00<00:00, 3020.93it/s]

100%|██████████| 70/70 [00:00<00:00, 3045.44it/s]

100%|██████████| 70/70 [00:00<00:00, 3044.36it/s]

100%|██████████| 70/70 [00:00<00:00, 2984.24it/s]

100%|██████████| 70/70 [00:00<00:00, 3054.50it/s]

100%|██████████| 70/70 [00:00<00:00, 2970.68it/s]

100%|██████████| 70/70 [00:00<00:00, 2993.46it/s]

100%|██████████| 70/70 [00:00<00:00, 3016.62it/s]

100%|██████████| 70/70 [00:00<00:00, 3005.59it/s]

100%|██████████| 70/70 [00:00<00:00, 3007.16it/s]

100%|██████████| 70/70 [00:00<00:00, 3029.32it/s]

100%|██████████| 70/70 [00:00<00:00, 3029.16it/s]

100%|██████████| 70/70 [00:00<00:00, 3001.08it/s]

100%|██████████| 70/70 [00:00<00:00, 2989.28it/s]

100%|██████████| 70/70 [00:00<00:00, 2985.94it/s]

100%|██████████| 70/70 [00:00<00:00, 2974.68it/s]

100%|██████████| 70/70 [00:00<

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
test_mae,█▅▄▃▂▁▃▂▂▂▂▂▁▂▁▂▂▂▁▂▁▂▂▂▂▂▂▂▁▂▂▂▃▂▂▃▂▂▂▂
test_mse,█▅▄▃▂▃▂▂▂▁▂▂▁▁▁▂▂▁▂▂▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▂▂▂▂▂
train_mae,█▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_mse,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,49
test_mae,0.00816
test_mse,0.00011
train_mae,0.00767
train_mse,9e-05


Training transforms (all default settings):   6%|▌         | 13/210 [03:23<52:41, 16.05s/it]/mnt/align3_drive/adrianoh/miniconda3/envs/python311/lib/python3.11/site-packages/wandb/sdk/wandb_run.py:2368: UserWarning: Run (4zvzgqqg) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),



100%|██████████| 70/70 [00:00<00:00, 3151.61it/s]

100%|██████████| 70/70 [00:00<00:00, 3100.00it/s]

100%|██████████| 70/70 [00:00<00:00, 3123.12it/s]

100%|██████████| 70/70 [00:00<00:00, 3097.16it/s]

100%|██████████| 70/70 [00:00<00:00, 2973.69it/s]

100%|██████████| 70/70 [00:00<00:00, 3024.92it/s]

100%|██████████| 70/70 [00:00<00:00, 2907.58it/s]

100%|██████████| 70/70 [00:00<00:00, 3086.45it/s]

100%|██████████| 70/70 [00:00<00:00, 3058.63it/s]

100%|██████████| 70/70 [00:00<00:00, 3076.71it/s]

100%|██████████| 70/70 [00:00<00:00, 2871.46it/s]

100%|██████████| 70/70 [00:00<00:00, 3024.67it/s]

100%|██████████| 70/70 [00:00<00:00, 3044.90it/s]

100%|██████████| 70/70 [00:00<00:00, 2843.73it/s]

100%|██████████| 70/70 [00:00<00:00, 3062.21it/s]

100%|██████████| 70/70 [00:00<00:00, 2914.45it/s]

100%|██████████| 70/70 [00:00<00:00, 3092.40it/s]

100%|██████████| 70/70 [00:00<00:00, 2925.22it/s]

100%|██████████| 70/70 [00:00<00:00, 3107.35it/s]

100%|██████████| 70/70 [00:00<

KeyboardInterrupt: 